# HW 5 
## Recommendation Systems

##Zaid Parvej Patel
##NetId: zp2090

## Part 1: Pyspark (40 Points)

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 27 kB/s 
     |████████████████████████████████| 198 kB 54.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=bd3fd7440b0ac5d8584aa8e261f432ba83ae0847b8e0419bee447566949cd1fe
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get

Now we authenticate a Google Drive client to download the filea we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [7]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [8]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



In [10]:
training.count()

80000

In [11]:
test.count()

20000

In [12]:
items.count()

1682

### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [13]:
# YOUR CODE HERE

In [14]:
import pandas as pd
import numpy as np

In [15]:
training_df = training.toPandas()

In [16]:
training_df

,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275


In [17]:
print("The number of ratings in the training dataset is",training_df['rating'].count())

The number of ratings in the training dataset is 80000


In [18]:
test_df = test.toPandas()

In [19]:
test_df

,user_id,item_id,rating,timestamp
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198
...,...,...,...,...
19995,458,648,4,886395899
19996,458,1101,4,886397931
19997,459,934,3,879563639
19998,460,10,3,882912371


In [20]:
print("The number of ratings in the test dataset is",test_df['rating'].count())

The number of ratings in the test dataset is 20000


In [21]:
movies_df = items.toPandas()

In [22]:
movies_df

,item_id,movie
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [23]:
print("The total number of movies in the dataset are",movies_df['item_id'].count())

The total number of movies in the dataset are 1682


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

maxIter = 5, regParam=0.01

In [24]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [25]:
# YOUR CODE HERE
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

Now compute the RMSE on the test dataset.


In [26]:
# YOUR CODE HERE

In [27]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("RMSE = " + str(rmse))

RMSE = 1.1322081487863145


At this point, you can use the trained model to produce the top-K recommendations for each user.  Recommend the top three movies for each user. 

In [28]:
# YOUR CODE HERE

In [29]:
top3movies = model.recommendForAllUsers(3)
top3movies_df = top3movies.toPandas()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [30]:
top3movies_df

,user_id,recommendations
0,1,"[(1131, 6.049376964569092), (904, 6.0154328346..."
1,3,"[(1311, 9.783595085144043), (700, 9.1908235549..."
2,5,"[(1176, 9.87546157836914), (1426, 8.6019039154..."
3,6,"[(718, 6.324886322021484), (919, 5.94831991195..."
4,9,"[(1131, 8.416128158569336), (1055, 6.893136501..."
...,...,...
938,938,"[(1131, 7.450057506561279), (842, 6.5268850326..."
939,940,"[(962, 5.460604667663574), (1615, 5.2934222221..."
940,941,"[(1167, 8.640521049499512), (733, 8.5658245086..."
941,942,"[(854, 6.363359451293945), (900, 6.01588678359..."


Print the name of the movies recommended for user 444  

In [31]:
# YOUR CODE HERE

In [32]:
user444 = top3movies_df[top3movies_df['user_id'] == 444]
x = user444['recommendations']
x.to_dict()
xnp = x.values
xlist = xnp.tolist()
item_ids =[]
for i in range(3):
  item_id = xlist[0][i][0]
  item_ids.append(item_id)
item_ids

[1001, 960, 1256]

In [33]:
user444_movies = pd.DataFrame()
d = {}
for item_id in item_ids:
  d.update({item_id:movies_df.loc[movies_df['item_id'] == item_id, 'movie'].item()})
user444_movies = (user444_movies.append(d, ignore_index = True)).T
user444_movies.reset_index(inplace=True)
user444_movies = user444_movies.rename(columns = {'index':'item_id',0:'movie'})
user444_movies

,item_id,movie
0,1001,"Stupids, The (1996)"
1,960,Naked (1993)
2,1256,"Designated Mourner, The (1997)"


## Part 2: Collaborative Filtering (60 Points)

scikit-surprise(http://surpriselib.com/) is a good library for multiple recommender systems algorithms. <br>You may start playing around with wrapper functions from the link provided.

Hint: Transform pyspark dataframe into pandas dataframe

In [34]:
# install surprise to build recommender in python
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 22.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630142 sha256=6fe9cb26c739fe82dc5e6f35743ed84a28cf1a2efc743e16b6852ab836b470c2
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


### Task. Memory-based Filtering 

Your task is to train a predictor using the `training` set provided above, and make predictions on the `test` set.

A. User-based recommendation

To make a prediction on user $u$'s rating on item $i$ ($R_{u, i}$), User-based recommendation finds the top-N user neighbors who have already completed rating on $i$, taking their average (unweighted or weighted by their similarity with $u$) as the prediction $\hat{R}_{u,i}$.


(1). Use default parameters, report *RMSE* on training & test set, respectively.

In [35]:
from surprise import KNNWithMeans
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from collections import defaultdict
from surprise import accuracy

In [36]:
# Your code here

In [37]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(training_df[['user_id', 'item_id', 'rating']], reader)
test_data = Dataset.load_from_df(test_df[['user_id', 'item_id', 'rating']], reader)

In [38]:
trainset = train_data.build_full_trainset()
algo = KNNWithMeans()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [39]:
testset_usingtrain = [train_data.df.loc[i].to_list() for i in range(len(train_data.df))]
predictions_train = algo.test(testset_usingtrain)

Calculate RMSE of the actual ratings $R$ and the predicted ratings $\hat{R}$ in the training set.

In [40]:
# Your code here

In [41]:
print("RMSE of the actual ratings and the predicted ratings in the training set =",accuracy.rmse(predictions_train))

RMSE: 0.7569
RMSE of the actual ratings and the predicted ratings in the training set = 0.7569412498224135


Now let's make predictions on the test set

In [42]:
# Your code here
testset_usingtest = [test_data.df.loc[i].to_list() for i in range(len(test_data.df))]
predictions_test = algo.test(testset_usingtest)

Calculate RMSE of the actual ratings $R$ and the predicted ratings $\hat{R}$ from the trained user-based recommendation.

In [43]:
# Your code here

In [44]:
print("RMSE of the actual ratings and the predicted ratings from the trained user-based recommendation =",accuracy.rmse(predictions_test))

RMSE: 0.9649
RMSE of the actual ratings and the predicted ratings from the trained user-based recommendation = 0.9649111798953492


(2). Now display the top 10 movies for user 10, ranked by the predicted rating scores in the test set.

In [45]:
# Your code here

In [46]:
def top_n_preds(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

In [47]:
top_n = top_n_preds(predictions_test, n=10)

In [48]:
top_n[10]

[(483, 5),
 (64, 4.979990947777017),
 (199, 4.965727090502444),
 (474, 4.916172802039486),
 (127, 4.90715526310589),
 (98, 4.881874481893698),
 (285, 4.8113268806185285),
 (603, 4.801331851179615),
 (488, 4.7867671367033955),
 (275, 4.7698543311004284)]

In [49]:
top_n = dict(top_n)
top_n_10 = top_n[10]

In [50]:
movie_ids_top10 = []
for i in range(len(top_n_10)):
  movie_ids_top10.append(top_n_10[i][0])
movie_ids_top10

[483, 64, 199, 474, 127, 98, 285, 603, 488, 275]

In [51]:
top_10_rec_df = pd.DataFrame()
d = {}
for item_id in movie_ids_top10:
  d.update({item_id:movies_df.loc[movies_df['item_id'] == item_id, 'movie'].item()})
top_10_rec_df = (top_10_rec_df.append(d, ignore_index = True)).T
top_10_rec_df.reset_index(inplace=True)
top_10_rec_df = top_10_rec_df.rename(columns = {'index':'item_id',0:'movie'})
top_10_rec_df

,item_id,movie
0,483,Casablanca (1942)
1,64,"Shawshank Redemption, The (1994)"
2,199,"Bridge on the River Kwai, The (1957)"
3,474,Dr. Strangelove or: How I Learned to Stop Worr...
4,127,"Godfather, The (1972)"
5,98,"Silence of the Lambs, The (1991)"
6,285,Secrets & Lies (1996)
7,603,Rear Window (1954)
8,488,Sunset Blvd. (1950)
9,275,Sense and Sensibility (1995)


(3). From what we learned in class, the number of nearest neighbors ($k$) considered for rating estimation $\hat{R}$ is an important hyperparameter affecting the prediction results. Repeat the training procedure above with different nearest neighbor selections (2-10), find the optimal $k$ in your experiment and report the corresponding *RMSE* in the test set.

In [52]:
# Your code here

In [54]:
k_rmse = {}
for k in range(2,11):
  algo = KNNWithMeans(k=k)
  algo.fit(trainset)
  predictions_test = algo.test(testset_usingtest)
  k_rmse.update({k:accuracy.rmse(predictions_test)})

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1250
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0712
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0418
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0247
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0129
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0043
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9974
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9924
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9892


In [55]:
k_rmse

{2: 1.125027351500927,
 3: 1.071152161449973,
 4: 1.0417979871306189,
 5: 1.0246540365073364,
 6: 1.0128837409113691,
 7: 1.004335995622638,
 8: 0.997356668570692,
 9: 0.9923535715146117,
 10: 0.9891847088270564}

Note that we can write a for-loop to iterate throuogh different choices, but scikit-surprise provides us with a simplified cross-validation interface (`surprise.model_selection.GridSearchCV`) to fine-tune such hyperparameter.


Report the optimal k value. 

Report the RMSE given the optimal k value

In [56]:
from surprise.model_selection import GridSearchCV

In [57]:
# Your code here

In [58]:
param_grid = {'k': [2,3,4,5,6,7,8,9,10]}
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=3, refit = True)
gs.fit(train_data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

From the above results, it can be observed that the RMSE for k=10 is comparatively lesser than for the other values of k. The RMSE on test set for model with k=10 yields to be 0.9854683982147456

B. **item-based recommendation**

To make a prediction on user $u$'s rating on item $i$ ($R_{u, i}$), Item-based recommendation finds the top-N item neighbors (the user has rated) to $i$, taking their average (unweighted or weighted by their similarity with $i$) as the prediction $\hat{R}_{u,i}$.



(1). Similar to the previous question, implement the item-based recommender systems trained on the  `training` set, report the *RMSE* on both the `training` and `test` set. (Note: apply the optimal $k$ obtained in last question.)

In [59]:
# Your code here

In [60]:
sim_options = {'user_based': False  } # compute  similarities between items 

In [61]:
algo = KNNWithMeans(k=10,sim_options = sim_options)
algo.fit(trainset)
ib_predictions_train = algo.test(testset_usingtrain)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [62]:
print("RMSE on trainset:",accuracy.rmse(ib_predictions_train))

RMSE: 0.5690
RMSE on trainset: 0.5690496491977741


In [63]:
algo = KNNWithMeans(k=10,sim_options = sim_options)
algo.fit(trainset)
ib_predictions_test = algo.test(testset_usingtest)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [64]:
print("RMSE on testset:",accuracy.rmse(ib_predictions_test))

RMSE: 0.9783
RMSE on testset: 0.9783018562689109


(2). Similar to previous question, display the top 10 movies for user 10, ranked by the predicted rating scores in the test set.

In [65]:
top_n = top_n_preds(ib_predictions_test, n=10)

In [66]:
top_n[10]

[(483, 4.941950958350294),
 (603, 4.919018203017838),
 (488, 4.84784489490977),
 (127, 4.80649558973847),
 (64, 4.8001987773405865),
 (285, 4.7896414388913735),
 (657, 4.758281084240285),
 (98, 4.747079109627615),
 (496, 4.7204290904022095),
 (474, 4.682108441858062)]

In [67]:
top_n = dict(top_n)
top_n_10 = top_n[10]

In [68]:
movie_ids_top10 = []
for i in range(len(top_n_10)):
  movie_ids_top10.append(top_n_10[i][0])
movie_ids_top10

[483, 603, 488, 127, 64, 285, 657, 98, 496, 474]

In [69]:
top_10_rec_df = pd.DataFrame()
d = {}
for item_id in movie_ids_top10:
  d.update({item_id:movies_df.loc[movies_df['item_id'] == item_id, 'movie'].item()})
top_10_rec_df = (top_10_rec_df.append(d, ignore_index = True)).T
top_10_rec_df.reset_index(inplace=True)
top_10_rec_df = top_10_rec_df.rename(columns = {'index':'item_id',0:'movie'})
top_10_rec_df

,item_id,movie
0,483,Casablanca (1942)
1,603,Rear Window (1954)
2,488,Sunset Blvd. (1950)
3,127,"Godfather, The (1972)"
4,64,"Shawshank Redemption, The (1994)"
5,285,Secrets & Lies (1996)
6,657,"Manchurian Candidate, The (1962)"
7,98,"Silence of the Lambs, The (1991)"
8,496,It's a Wonderful Life (1946)
9,474,Dr. Strangelove or: How I Learned to Stop Worr...


(3). Given the same number of nearest neighbor ($k$), compare and discuss the user-based and item-based recommendation, which performs better on the test set?

Considering the same numbers of nearest neighbors that is k=10 and the respective RMSE scores for both the content-based and item-based recommendation systems that are as follows:


*   RMSE on test set for User Based  = 0.98546839821474567
*   RMSE on test set for Item Based = 0.978301856268910


Looking at the above RMSE values obtained by both the methods of recommendation on test set, we can conclude that Item Based Recommendation System is performing better with respect to User Based as it has a lower RMSE value.